## Extracting info on all listings with first air date from 1990 using the MAL api

In [ ]:
!pip install requests

In [1]:
import numpy as np
import pandas as pd
import requests
import datetime

In [2]:
CLIENT_ID = '02d2f7a9a2ca225cf0c46ca312603669'

In [3]:
##EXAMPLE
#url = 'https://api.myanimelist.net/v2/anime/50612?fields=rank,mean,genres,background,media_type'
##sort list by number of users
url = 'https://api.myanimelist.net/v2/anime/season/2023/summer?sort=anime_num_list_users&limit=50&fields=start_season'
#url = 'https://api.myanimelist.net/v2/users/AnarchistSasuke/animelist?fields=list_status&limit=10'
response = requests.get(url, headers = {
        'X-MAL-CLIENT-ID': '02d2f7a9a2ca225cf0c46ca312603669'})
response.json()

{'data': [{'node': {'id': 21,
    'title': 'One Piece',
    'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/6/73245.jpg',
     'large': 'https://cdn.myanimelist.net/images/anime/6/73245l.jpg'},
    'start_season': {'year': 1999, 'season': 'fall'}}},
  {'node': {'id': 51009,
    'title': 'Jujutsu Kaisen 2nd Season',
    'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/1792/138022.jpg',
     'large': 'https://cdn.myanimelist.net/images/anime/1792/138022l.jpg'},
    'start_season': {'year': 2023, 'season': 'summer'}}},
  {'node': {'id': 54112,
    'title': 'Zom 100: Zombie ni Naru made ni Shitai 100 no Koto',
    'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/1384/136408.jpg',
     'large': 'https://cdn.myanimelist.net/images/anime/1384/136408l.jpg'},
    'start_season': {'year': 2023, 'season': 'summer'}}},
  {'node': {'id': 235,
    'title': 'Detective Conan',
    'main_picture': {'medium': 'https://cdn.myanimelist.net/image

In [ ]:
## TOP ANIME EXAMPLE
url = 'https://api.myanimelist.net/v2/anime/ranking?ranking_type=all'+'&limit=500'
response = requests.get(url, headers = {
       'X-MAL-CLIENT-ID': '02d2f7a9a2ca225cf0c46ca312603669'})
response = response.json()['data']

df = pd.DataFrame(columns =['anime_id','anime_name'])
for top_anime in response:
  anime_id = top_anime['node']['id']
  anime_name = top_anime['node']['title']
  adis = pd.DataFrame([[anime_id, anime_name]], columns =['anime_id','anime_name'])
  df = pd.concat([df,adis], ignore_index = True)

print(df)

In [ ]:
##SEASONAL EXAMPLE
year="2020"
season = "spring"
seasonal_url = 'https://api.myanimelist.net/v2/anime/season/2023/summer?sort=anime_num_list_users&limit=50&fields=start_season'
response = requests.get(seasonal_url, headers = {
        'X-MAL-CLIENT-ID': '02d2f7a9a2ca225cf0c46ca312603669'})
response = response.json()['data']

print(response[0]['node']['start_season']['season'])
print('fall')


In [21]:
##START HERE P1
sdf = pd.DataFrame(columns =['anime_id','anime_name', 'anime_start_year'])
year=1990
season = pd.Series(["winter", "spring", "summer", "fall"])
nyear = int(datetime.date.today().year)
nmonth = datetime.date.today().month

if nmonth <= 3:
    nseason = "winter"
elif nmonth <= 6:
    nseason = "spring"
elif nmonth <= 9:
    nseason = "summer"
else:
    nseason = "fall"

In [22]:
## PT 2
while year <= nyear:
    for s in season:
        seasonal_url = 'https://api.myanimelist.net/v2/anime/season/'+str(year)+'/'+s+ '?sort=anime_num_list_users&limit=500&fields=start_season'
        response = requests.get(seasonal_url, headers = {'X-MAL-CLIENT-ID': '02d2f7a9a2ca225cf0c46ca312603669'})
        response = response.json()['data']
        for top_sanime in response:
            anime_id = top_sanime['node']['id']
            anime_name = top_sanime['node']['title']
            anime_start_year = int(top_sanime['node']['start_season']['year'])
            adis = pd.DataFrame([[anime_id, anime_name, anime_start_year]], columns =['anime_id','anime_name', 'anime_start_year'])
            sdf = pd.concat([sdf,adis], ignore_index = True)
        if year == nyear and s == nseason:
            break
    year = year+1

In [29]:
## PT 3: remove dups and earlier shows
year=1990
sdf=sdf.loc[sdf['anime_start_year'] >= year]
sdf.drop_duplicates(inplace = True)
sdf.reset_index(inplace = True, drop=True)

In [28]:
## PT 4
df_info = pd.DataFrame(columns =['anime_id','anime_name', 'anime_start_year','synopsis', 'num_list_users', 'nsfw', 'num_episodes', 'rating', 'media_type', 'genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7'])
df_info.drop(df_info.index, inplace=True)
count = 0

In [781]:
## segment the entries for years after 2009
year = 2023
df_temp = sdf.loc[sdf['anime_start_year'] == year]
temp = df_temp.iloc[600:]

In [782]:
## PT 5 repeat this until year == current year
#year = 2009
#temp = sdf.loc[sdf['anime_start_year'] == year]

for ind in temp.index:
    adis = []
    anime_id = temp['anime_id'][ind]
    url = 'https://api.myanimelist.net/v2/anime/'+str(anime_id)+'?fields=id,title,synopsis,num_list_users,nsfw,num_episodes,rating,genres,media_type'
    details = requests.get(url, headers = {
          'X-MAL-CLIENT-ID': '02d2f7a9a2ca225cf0c46ca312603669'})
    details = details.json()
    adis = [anime_id, temp['anime_name'][ind], temp['anime_start_year'][ind]]
    fields = details.keys()
    #new
    if 'synopsis' in fields:
        adis.append(details['synopsis'].replace('\n', '').replace('[Written by MAL Rewrite]', ''))
    else:
        adis.append('na')
    #new
    adis.append(details['num_list_users'])
    adis.append(details['nsfw'])
    adis.append(details['num_episodes'])
    if 'rating' in fields:
        adis.append(details['rating'])
    else:
        adis.append('na')
    adis.append(details['media_type'])
    if 'genres' in fields:
        genres = details['genres']
        for i in range(0,7):
            if i in range(len(genres)):
                adis.append(genres[i]['name'])
            else:
                adis.append('na')
    else:
        adis.extend(['na', 'na', 'na', 'na', 'na', 'na', 'na'])
    #print(adis)
    df_info.loc[count] = adis
    count = count+1


In [750]:
## gives how many shows in a given year
len(sdf.loc[sdf['anime_start_year'] == 2023])

715

In [786]:
## checks how many entries have start date before a certain year
len(sdf.loc[sdf['anime_start_year'] <= 2023])

16731

In [783]:
## check the number of entries in df_info
count = len(df_info)
count

16731

In [787]:
## run this to reset to a certain year
##df_info = df_info.loc[df_info['anime_start_year'] <= 2018]
df_info

,anime_id,anime_name,anime_start_year,synopsis,num_list_users,nsfw,num_episodes,rating,media_type,genre1,genre2,genre3,genre4,genre5,genre6,genre7
0,895,Dragon Ball Z Movie 02: Kono Yo de Ichiban Tsu...,1990,"In his laboratory, biotechnology expert Dr. Ko...",111205,white,1,pg_13,movie,Action,Adventure,Comedy,Fantasy,Sci-Fi,Shounen,na
1,2355,Devilman: Yochou Sirene-hen,1990,Demons exist and their power is beyond what hu...,22375,white,1,r+,ova,Gore,Horror,Mythology,Sci-Fi,Shounen,Supernatural,na
2,1375,Watashi no Ashinaga Ojisan,1990,Daddy Long Legs is based on the novel of the s...,16367,white,40,g,tv,Comedy,Historical,Romance,School,Slice of Life,na,na
3,1261,Kyattou Ninden Teyandee,1990,"In the city of Edoropolis, hundreds of mechani...",13370,white,54,g,tv,Action,Anthropomorphic,Comedy,Mecha,Sci-Fi,Super Power,na
4,951,Chibi Maruko-chan,1990,Momoko Sakura is an elementary school student ...,11986,white,142,g,tv,Comedy,Shoujo,Slice of Life,na,na,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16726,56622,Yankee Hamster Season 4,2023,The fourth season of Yankee Hamster.,54,white,0,pg_13,ona,Comedy,na,na,na,na,na,na
16727,56623,Nui Story,2023,,49,white,0,pg_13,ona,Comedy,na,na,na,na,na,na
16728,56821,Yoru wo Matsu yo,2023,Music video for the song Yoru wo Matsu yo by M...,36,white,1,g,music,Music,na,na,na,na,na,na
16729,56820,Sanshiliu Qi,2023,,33,white,13,pg_13,ona,Action,Drama,Historical,Martial Arts,na,na,na


In [788]:
df_info.to_csv("C://Users/annad/OneDrive/Documents/JNote/animes90to23.csv")